In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import seaborn as sns
from Bio import SeqIO, Seq

import glob, os, yaml, subprocess, itertools, sparse, vcf
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from statsmodels.distributions.empirical_distribution import ECDF
import sklearn.metrics
from sklearn.decomposition import PCA
import timeit
import scipy.stats as st

who_variants = pd.read_csv("/n/data1/hms/dbmi/farhat/Sanjana/MIC_data/WHO_resistance_variants_all.csv")

# Numbers of Isolates and Features dropped at different thresholds

In [2]:
drug = "Pyrazinamide"
model_prefix = "tiers=1_drop_HET_noPopCorrect"

prefilt = pd.read_pickle(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/Drugs/{drug}/{model_prefix}/prefilt_matrix.pkl")
filt = pd.read_pickle(f"/n/data1/hms/dbmi/farhat/ye12/who/analysis/Drugs/{drug}/{model_prefix}/filt_matrix.pkl")
print(prefilt.shape)
print(filt.shape)

(17756, 1242)
(17613, 1242)


In [3]:
def print_threshold_summaries(prefilt, missing_thresh=None):
    
    # drop all isolates with more than 1 missing feature
    if missing_thresh is None:
        filt_isolates = prefilt.dropna(axis=0, thresh=prefilt.shape[1]-1)
    else:
        filt_isolates = prefilt.dropna(axis=0, thresh=(1-missing_thresh)*prefilt.shape[1])
        
    # remove all features with ANYTHING missing
    filt_feat = filt_isolates.dropna(axis=1)
    print(filt_feat.shape)
    print(f"Dropped {prefilt.shape[0] - filt_isolates.shape[0]} out of {prefilt.shape[0]} isolates and {prefilt.shape[1] - filt_feat.shape[1]} out of {prefilt.shape[1]} features")

In [10]:
kwargs = yaml.safe_load(open("config.yaml"))

tiers_lst = kwargs["tiers_lst"]
drug = kwargs["drug"]
out_dir = kwargs["out_dir"]
model_prefix = kwargs["model_prefix"]
missing_thresh = kwargs["missing_thresh"]
het_mode = kwargs["het_mode"]
af_thresh = kwargs["AF_thresh"]
include_synonymous = kwargs["include_synonymous"]

print("Tiers for this model:", tiers_lst)

if not os.path.isdir(os.path.join(out_dir, drug)):
    print(f"Creating output directory {os.path.join(out_dir, drug)}")
    os.mkdir(os.path.join(out_dir, drug))
    
if not os.path.isdir(os.path.join(out_dir, drug, model_prefix)):
    print(f"Creating output directory {os.path.join(out_dir, drug, model_prefix)}")
    os.mkdir(os.path.join(out_dir, drug, model_prefix))

genos_dir = '/n/data1/hms/dbmi/farhat/ye12/who/full_genotypes'
phenos_dir = '/n/data1/hms/dbmi/farhat/ye12/who/phenotypes'
phenos_dir = os.path.join(phenos_dir, f"drug_name={drug}")


############# STEP 1: GET ALL AVAILABLE PHENOTYPES #############


dfs_list_phenos = []

for fName in os.listdir(phenos_dir):
    dfs_list_phenos.append(pd.read_csv(os.path.join(phenos_dir, fName)))

df_phenos = pd.concat(dfs_list_phenos)

# check that there are no duplicated phenotypes
assert len(df_phenos) == len(df_phenos.sample_id.unique())

# check that there is resistance data for all samples
assert sum(pd.isnull(df_phenos.phenotype)) == 0
    
print(f"{len(df_phenos)} samples with phenotypes for {drug}")


############# STEP 2: GET ALL AVAILABLE GENOTYPES #############


# first get all the genotype files associated with the drug
geno_files = []

for subdir in os.listdir(os.path.join(genos_dir, f"drug_name={drug}")):
    
    # subdirectory (tiers)
    full_subdir = os.path.join(genos_dir, f"drug_name={drug}", subdir)

    # the last character is the tier number
    if subdir[-1] in tiers_lst:
        
        for fName in os.listdir(full_subdir):
            
            # some hidden files (i.e. Git files) are present, so ignore them
            if fName[0] != ".":
                geno_files.append(os.path.join(full_subdir, fName))
          
        
dfs_lst = []
for i, fName in enumerate(geno_files):
        
    print(f"Working on dataframe {i+1}/{len(geno_files)}")
    print(fName)

    # read in the dataframe
    df = pd.read_csv(fName)

    df_avail_isolates = df.loc[df.sample_id.isin(df_phenos.sample_id)]
    
    # keep all variants
    if include_synonymous:
        dfs_lst.append(df_avail_isolates)
    else:
        # keep only 1) noncoding variants and 2) non-synonymous variants in coding regions. 
        # P = coding variants, C = synonymous or upstream variants (get only upstream variants by getting only negative positions), and N = non-coding variants on rrs/rrl
        dfs_lst.append(df_avail_isolates.loc[((df_avail_isolates.category.astype(str).str[0] == 'c') & (df_avail_isolates.category.str.contains('-'))) | 
                                             (df_avail_isolates.category.astype(str).str[0].isin(['n', 'p']))])
        
df_model = pd.concat(dfs_lst)

Tiers for this model: ['1']
22743 samples with phenotypes for Pyrazinamide
Working on dataframe 1/3
/n/data1/hms/dbmi/farhat/ye12/who/full_genotypes/drug_name=Pyrazinamide/tier=1/run-1660066839545-part-r-00036
Working on dataframe 2/3
/n/data1/hms/dbmi/farhat/ye12/who/full_genotypes/drug_name=Pyrazinamide/tier=1/run-1660066839545-part-r-00055
Working on dataframe 3/3
/n/data1/hms/dbmi/farhat/ye12/who/full_genotypes/drug_name=Pyrazinamide/tier=1/run-1660066839545-part-r-00046


In [30]:
dfs_lst[0]

,sample_id,gene_symbol,category,variant_allele_frequency,variant_binary_status
1,57075,pncA,p.Val130Met,0.0,0.0
2,64839,pncA,c.-363_-37del,0.0,0.0
3,57075,pncA,p.His57Gln,0.0,0.0
4,164473,pncA,p.Val155Leu,0.0,0.0
6,64839,pncA,p.His137Arg,0.0,0.0
...,...,...,...,...,...
19307760,139790,pncA,p.Val180Phe,0.0,0.0
19307761,55356,pncA,p.Arg148Cys,0.0,0.0
19307763,55356,pncA,p.Asn11Tyr,0.0,0.0
19307764,139790,pncA,p.His82Arg,0.0,0.0


In [32]:
df_phenos.category.unique()

array(['WHO_current'], dtype=object)

In [11]:
df_model

,sample_id,gene_symbol,category,variant_allele_frequency,variant_binary_status
1,57075,pncA,p.Val130Met,0.0,0.0
2,64839,pncA,c.-363_-37del,0.0,0.0
3,57075,pncA,p.His57Gln,0.0,0.0
4,164473,pncA,p.Val155Leu,0.0,0.0
6,64839,pncA,p.His137Arg,0.0,0.0
...,...,...,...,...,...
5605235,164459,panD,c.-400_-35del,0.0,0.0
5605237,164459,panD,c.-588_-25del,0.0,0.0
5605239,164459,panD,p.Thr92Met,0.0,0.0
5605243,164459,panD,p.Lys7_Val106del,0.0,0.0


In [14]:
len(df_model.sample_id.unique())

17961

In [22]:
variants_df = pd.DataFrame(variants).rename(columns={0:"category"})

In [29]:
variants_df.loc[(variants_df.category.astype(str).str[0] == 'p') & (variants_df.category.str.contains("_"))]

,category
12,p.Asp129_Val131del
16,p.Asp12_Phe13insAlaSerGluProProSerGlnLysSer
20,p.Asp129_Val131dup
45,p.Gly24_Ile133del
51,p.Thr22_Thr160del
...,...
1131,p.Glu522_Thr660del
1135,p.Ser546_Pro772del
1191,p.Lys7_Val106del
1201,p.Arg99_Thr100insSer


In [15]:
variants = df_model.category.unique()

In [13]:
df_model.loc[df_model.gene_symbol.str.contains("lof", case=False)]

,sample_id,gene_symbol,category,variant_allele_frequency,variant_binary_status


In [18]:
df_model.loc[df_model.category.str.contains("fs", case=False)]

,sample_id,gene_symbol,category,variant_allele_frequency,variant_binary_status


# Unused Functions

In [4]:
def simulate_null_regression(var_idx, num_bootstrap=100):
        
    # make variant of interest all 0 so that its coefficient will be 0
    df = model_inputs.copy()
    if "sample_id" in df.columns:
        df = df.set_index("sample_id")
    df.iloc[:, var_idx] = 0
    
    # concatenate the eigenvectors to the matrix
    X = np.concatenate([df.values, eigenvec_df.values], axis=1)

    # scale inputs
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    assert len(y) == X.shape[0]
    
    coef_lst = []
    for _ in range(num_bootstrap):
        null_model = LogisticRegressionCV(Cs=1/np.logspace(-4, 4, 9), 
                                 cv=5,
                                 penalty='l2', 
                                 max_iter=10000, 
                                 multi_class='ovr',
                                 scoring='neg_log_loss'
                                )

        null_model.fit(X, y)
        coef_lst.append(np.squeeze(null_model.coef_)[var_idx])
    return coef_lst


In [5]:
def get_pvalues_add_ci(baseline_df, bs_df, col, ci=95):
    
    alpha = (100-ci)/100
    baseline_df[col] = baseline_df[col].astype(str)
    pvals = []
    for i, pos in enumerate(baseline_df[col].values):
        
        # get the percentile of 0
        ecdf = ECDF(bs_df[pos].values)
        zero_percentile = ecdf(0)
        
        # if the percentile is greater than 0.5, then the hypothesis test is for probability of lying in the upper tail (i.e. > 97.5 for a 95% CI)
        # if the percentile i less than 0.5, then the hypothesis test is for probability of lying in the lower tail (i.e. < 2.5 for a 95% CI)
        if zero_percentile > 1 or zero_percentile < 0:
            print(pos)
        
        if zero_percentile < 0.5:
            pvals.append(2 * zero_percentile)
        else:
            pvals.append(2 * (1 - zero_percentile))
            
        # add confidence intervals
        diff = (100-ci)/2
        lower, upper = np.percentile(bs_df[pos].values, q=(diff, 100-diff))
        baseline_df.loc[i, "Lower_CI"] = lower
        baseline_df.loc[i, "Upper_CI"] = upper
        
    pvals = np.array(pvals)
    return pvals, alpha

In [8]:
def volcano_plot(baseline_df, bs_df, col, pvals, ci=95, exclude_lst=[], label_thresh=0, title_suffix="", saveFig=None):
        
    # replace 0s so that log can be taken for plotting
    if len(np.unique(pvals)) > 1:
        min_non_zero = np.sort(np.unique(pvals))[1]
    else:
        min_non_zero = 1e-3
    pvals[pvals == 0] = min_non_zero
    
    baseline_df["neg_log_pval"] = -np.log(pvals)
    baseline_df["pval_significant"] = (baseline_df["pval"] < alpha).astype(int)
    
    # verify input order
    assert len(bs_df.columns) == len(set(bs_df.columns).intersection(baseline_df[col].values))
    
    # only plot features with nonzero coefficients
    bs_df_positive = baseline_df.query("coef > 0")
    
    fig, ax = plt.subplots()
    sns.scatterplot(data=baseline_df.loc[~baseline_df[col].isin(exclude_lst)], x="coef", y="neg_log_pval", 
                    alpha=0.75,
                    hue="pval_significant", 
                    s=30,
                    palette={1:"darkred", 0:"lightgray"},
                    ax=ax)

    ax.legend().set_visible(False)
    #plt.xscale("log")
    plt.title("Ridge Regression Coefficients and Bootstrapped\np-values" + title_suffix + f", α = {alpha}")
    sns.despine()
    
    if saveFig is not None:
        plt.savefig(saveFig, dpi=300, bbox_inches="tight")
    else:
        plt.show()